In [2]:
from pathlib import Path
import park_and_ride_analysis

module_dir = Path(park_and_ride_analysis.__file__).parent
project_dir = module_dir.parent.parent
data_dir = project_dir / "data"

processed_data_dir = data_dir / "processed"

output_dir = project_dir / "output"
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
from geopy.geocoders import Nominatim

def get_coordinates(zip_code, country="Germany"):
    geolocator = Nominatim(user_agent="zip_code_locator")
    location = geolocator.geocode(f"{zip_code}, {country}")
    
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

In [4]:
import folium

darmstadt_coords = (49.8728, 8.6512)
map = folium.Map(location=darmstadt_coords, zoom_start=10)

In [ ]:
# Read ZIP Codes from a csv file at data_dir/raw/survey_tt.csv at the field "PLZ" and for every zip code place a marker on the map

import pandas as pd
zip_code_count = {}

df = pd.read_csv(f"{data_dir}/raw/survey_tt.csv", sep=";")
for index, row in df.iterrows():
    if pd.isna(row['PLZ']):
        continue
    
    row['PLZ'] = int(row['PLZ'])
    coords = get_coordinates(row['PLZ'])
    if coords is None:
        continue
    
    plz = row['PLZ']
    if plz in zip_code_count:
        zip_code_count[plz] += 1
    else:
        zip_code_count[plz] = 1
    
    # Marker size and color based on occurrence count
    marker_size = 5 + (zip_code_count[plz] * 3)
    color = 'blue' if zip_code_count[plz] == 1 else 'red' if zip_code_count[plz] == 2 else 'green'
    
    folium.CircleMarker(
        location=[coords[0], coords[1]],
        radius=marker_size,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"{plz}: {zip_code_count[plz]} occurrence(s)"
    ).add_to(map)


map.save(output_dir / 'zip_map_commuters.html')

In [24]:
# Read the "Adresse" Column at data_dir/raw/zielorte.csv and use a python package to search for the coordinates and put them in a list 
import pandas as pd

df = pd.read_csv(f"{data_dir}/raw/zielorte.csv", sep=";")
addresses = df["Adresse"].tolist()
map = folium.Map(location=darmstadt_coords, zoom_start=14)

address_count = {}

# List of colors to map to occurrences
colors = ['blue', 'red', 'green', 'yellow', 'orange', 'purple', 'pink', 'brown', 'gray', 'cyan', 'magenta', 'lime', 'maroon', 'navy', 'olive']


geolocator = Nominatim(user_agent="myGeolocator")

for address in addresses:    
    if address in address_count:
        address_count[address] += 1
    else:
        address_count[address] = 1

    # Marker size and color based on occurrence count

    marker_size = 5 + (address_count[address] * 2)

    # set the color dynamically based on the number of occurrences of the address
 
    #  Get the number of occurrences for the address
    occurrences = address_count[address]

    # Determine the color based on the number of occurrences, using modulo to wrap around if needed
    color = colors[(occurrences - 1) % len(colors)]




for address in address_count:
    print(f"{address}: {address_count[address]}")
    location = geolocator.geocode(address)
    occurrences = address_count[address]
    color = colors[(occurrences - 1) % len(colors)]
    marker_size = 5 + (occurrences * 2)

    folium.CircleMarker(
        location=[location.latitude, location.longitude],
        radius=marker_size,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"{address}: {address_count[address]} occurrence(s)"
    ).add_to(map)

# add a legend to the map that indicates the color of the markers and the number of occurrences

legend_html = """
<div style="position: fixed; 
     bottom: 100px; left: 450px; width: 150px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white;
     opacity: 0.8;
     ">
     &nbsp; Address Occurrences <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:blue"></i> 1 <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:yellow"></i> 4 (TZ Rhein-Main) <br>
    &nbsp; <i class="fa fa-circle fa-1x" style="color:purple"></i> 6 (Luisenplatz) <br>
    &nbsp; <i class="fa fa-circle fa-1x" style="color:navy"></i> 14 (TU Stadtmitte) <br>

</div>
"""
map.get_root().html.add_child(folium.Element(legend_html))


map.save(output_dir / 'address_map_commuters.html')



Karolinenplatz 5 64289 Darmstadt: 14
Deutsche-Telekom-Allee 64295 Darmstadt: 4
Alsfelder Str. 45 64289 Darmstadt: 1
Franziska-Braun-Straße 10 64287 Darmstadt: 1
Luisenplatz 64283 Darmstadt: 6
Am Hbf 20 64293 Darmstadt: 1
Beckstraße 44 64287 Darmstadt: 1
Nieder-Ramstädter Str. 170 64285 Darmstadt: 1
Kasinostraße 129 64293 Darmstadt: 1


In [44]:
import pandas as pd
import folium
from collections import defaultdict
from geopy.geocoders import Nominatim

# Read the CSV file
df = pd.read_csv(f"{data_dir}/raw/park_and_ride.csv", sep=";")


# Create a dictionary with the count of each coordinate and the name of the location
location_dict = defaultdict(lambda: {"count": 0, "name": None})

for _, row in df.iterrows():
    coord = row["coordinates"]
    name = row["P+R"]
    location_dict[coord]["count"] += 1
    location_dict[coord]["name"] = name

# Coordinates for Darmstadt (replace with actual coordinates if different)
darmstadt_coords = [49.8728, 8.6512]

# Initialize a Folium map
map = folium.Map(location=darmstadt_coords, zoom_start=12.3)

# Place a marker on the map for every entry in the dictionary
for coord, data in location_dict.items():
    count = data["count"]
    name = data["name"]
    coords = coord.split(",")
    marker_size = 5 + (count * 2)
    color = 'blue' if count == 1 else 'red' if count == 2 else 'green'
    folium.CircleMarker(
        location=[float(coords[0]), float(coords[1])],
        radius=marker_size,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"{name}: {count} occurrence(s)"
    ).add_to(map)

    folium.Marker(
        location=[float(coords[0]), float(coords[1])],
        icon=folium.DivIcon(
            html=f"""
                <div style="
                    font-size: 8pt; 
                    color: black; 
                    background-color: white; 
                    padding: 2px; 
                    border-radius: 3px; 
                    white-space: nowrap; 
                    display: inline-block; 
                    width: auto;
                    transform: translate(20px, -5px);
                    ">
                    {name}
                </div>"""
        )
    ).add_to(map)


legend_html = """
<div style="position: fixed; 
     bottom: 100px; left: 450px; width: 150px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white;
     opacity: 0.8;
     ">
     &nbsp; Address Occurrences <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:blue"></i> 1 <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i> 2  <br>
    &nbsp; <i class="fa fa-circle fa-1x" style="color:green"></i> 3 <br>
</div>
"""
map.get_root().html.add_child(folium.Element(legend_html))


map.save(output_dir / 'park_and_ride_map_commuters.html')
